In [9]:
# Data
import numpy as np

# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}
    
    X = [char_to_idx[x] for x in txt]
    X = np.array(X)
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [13]:
# Model
import impl.layer as l

class RNN:

    def __init__(self, D, H, L, char2idx, idx2char, p_dropout):
        self.D = D
        self.H = H
        self.L = L
        self.char2idx = char2idx
        self.idx2char = idx2char
        self.vocab_size = len(char2idx)
        self.losses = {'train':[], 'smooth train':[]}
        self.p_dropout = p_dropout
        
        # Model parameters
        m = dict(
            Wxh=np.random.randn(D, H) / np.sqrt(D / 2.),
            Whh=np.random.randn(H, H) / np.sqrt(H / 2.),
            Why=np.random.randn(H, D) / np.sqrt(H / 2.),
            bh=np.zeros((1, H)),
            by=np.zeros((1, D))
            )
        self.model = []
        for _ in range(self.L):
            self.model.append(m)
            
    def initial_state(self):
        return np.zeros((1, self.H))
    
    def dropout_forward(self, X, p_dropout):
        u = np.random.binomial(1, p_dropout, size=X.shape) / p_dropout
        #         u = np.random.binomial(1, q, size=X.shape)
        out = X * u
        cache = u
        return out, cache

    def dropout_backward(self, dout, cache):
        dX = dout * cache
        return dX

    def forward(self, X, h, m, train):
        Wxh, Whh, Why = m['Wxh'], m['Whh'], m['Why']
        bh, by = m['bh'], m['by']

        hprev = h.copy()
        X_one_hot = X.copy()
    
        X = (X_one_hot @ Wxh) + (hprev @ Whh) + bh
        h, h_cache = l.tanh_forward(X)
        
        y, y_cache = l.fc_forward(h, Why, by)

        # Dropout for training
        if train:
            y, y_do_cache = self.dropout_forward(X=y, p_dropout=self.p_dropout)
            cache = (X_one_hot, Wxh, hprev, Whh, h_cache, y_cache, y_do_cache)
        else:
            cache = (X_one_hot, Wxh, hprev, Whh, h_cache, y_cache)

        return y, h, cache

    def backward(self, dy, dh, cache):
        #         if train: 
        # Backward is performed for learning/training and not testing or validation.
        X_one_hot, Wxh, hprev, Whh, h_cache, y_cache, y_do_cache = cache
        dy = self.dropout_backward(dout=dy, cache=y_do_cache)

        dh_next = dh.copy()
        
        # Hidden to output gradient
        dh, dWhy, dby = l.fc_backward(dy, y_cache)
        dh += dh_next
        dby = dby.reshape((1, -1))

        # tanh
        dX_one_hot = l.tanh_backward(dh, h_cache)

        # Hidden gradient
        dbh = dX_one_hot * 1.0
        dWhh = hprev.T @ dX_one_hot
        dWxh = X_one_hot.T @ dX_one_hot
        
        dX = dX_one_hot @ Wxh.T
        dh = dX_one_hot @ Whh.T

        grad = dict(Wxh=dWxh, Whh=dWhh, Why=dWhy, bh=dbh, by=dby)
        
        return dX, dh, grad

    def train_forward(self, X_train, h):
        ys, caches = [], []
        h_init = h.copy()
        h = []
        for _ in range(self.L):
            h.append(h_init.copy())
            caches.append([])
            
        for X in X_train:
            X_one_hot = np.zeros(self.D)
            X_one_hot[X] = 1.
            X = X_one_hot.reshape(1, -1)
            for layer in range(self.L):
                y, h[layer], cache = self.forward(X, h[layer], self.model[layer], train=True)
                caches[layer].append(cache)
                X = y.copy() # the output for the previous layer is the input for the next layer
            ys.append(y)
            
        return ys, caches

    def cross_entropy(self, y_pred, y_train):
        m = y_pred.shape[0]

        prob = l.softmax(y_pred)
        log_like = -np.log(prob[range(m), y_train])
        data_loss = np.sum(log_like) / m

        return data_loss

    def dcross_entropy(self, y_pred, y_train):
        m = y_pred.shape[0]

        grad_y = l.softmax(y_pred)
        grad_y[range(m), y_train] -= 1.0
        grad_y /= m

        return grad_y
    
    def loss_function(self, y_train, ys):
        loss, dys = 0.0, []

        for y_pred, y in zip(ys, y_train):
            loss += self.cross_entropy(y_pred, y)
            dy = self.dcross_entropy(y_pred, y)
            dys.append(dy)
            
        return loss, dys

    def train_backward(self, dys, caches):
        dh, grad, grads = [], [], []
        for layer in range(self.L):
            dh.append(np.zeros((1, self.H)))
            grad.append({key: np.zeros_like(val) for key, val in self.model[layer].items()})
            grads.append({key: np.zeros_like(val) for key, val in self.model[layer].items()})
        
        dXs = []
        for t in reversed(range(len(dys))):
            dy = dys[t]
            for layer in reversed(range(self.L)):
                dX, dh[layer], grad[layer] = self.backward(dy, dh[layer], caches[layer][t]) # train=True
                for key in grad[layer].keys():
                    grads[layer][key] += grad[layer][key]
                dy = dX.copy() # The input for the next layer is the output for the previous layer
            dXs.append(dX)
            
        return dXs, grads
    
    def test(self, X_seed, h, size):
        chars = [self.idx2char[X_seed]]
        idx_list = list(range(self.vocab_size))
        X = X_seed
        
        h_init = h.copy()
        h = []
        for _ in range(self.L):
            h.append(h_init.copy())

        for _ in range(size):
            X_one_hot = np.zeros(self.D)
            X_one_hot[X] = 1.
            X = X_one_hot.reshape(1, -1)
            for layer in range(self.L):
                y, h[layer], _ = self.forward(X, h[layer], self.model[layer], train=False)
                X = y.copy()
                
            prob = l.softmax(y)
            idx = np.random.choice(idx_list, p=prob.ravel())
            chars.append(self.idx2char[idx])
            X = idx

        return ''.join(chars)

In [14]:
def get_minibatch(X, y, minibatch_size, shuffle):
    minibatches = []

    for i in range(0, X.shape[0], minibatch_size):
    # for i in range(0, X.shape[0] - minibatch_size + 1, 1):
        X_mini = X[i:i + minibatch_size]
        y_mini = y[i:i + minibatch_size]
        minibatches.append((X_mini, y_mini))

    return minibatches

def adam_rnn(nn, X_train, y_train, alpha, mb_size, n_iter, print_after):
    M, R = [], []
    for layer in range(nn.L):
        M.append({key: np.zeros_like(val) for key, val in nn.model[layer].items()})
        R.append({key: np.zeros_like(val) for key, val in nn.model[layer].items()})
        
    beta1 = .99
    beta2 = .999
    state = nn.initial_state()
    eps = 1e-8
    smooth_loss = 1.
    minibatches = get_minibatch(X_train, y_train, mb_size, shuffle=False)

    # Epochs
    for iter in range(1, n_iter + 1):

        # No full batch or files
        # Minibatches
        for idx in range(len(minibatches)):
            X_mini, y_mini = minibatches[idx]
            ys, caches = nn.train_forward(X_mini, state)
            loss, dys = nn.loss_function(y_mini, ys)
            _, grads = nn.train_backward(dys, caches)
            nn.losses['train'].append(loss)
            smooth_loss = (0.999 * smooth_loss) + (0.001 * loss)
            nn.losses['smooth train'].append(smooth_loss)

            for layer in range(nn.L):
                for key in grads[layer].keys(): #key, value: items
                    M[layer][key] = l.exp_running_avg(M[layer][key], grads[layer][key], beta1)
                    R[layer][key] = l.exp_running_avg(R[layer][key], grads[layer][key]**2, beta2)

                    m_k_hat = M[layer][key] / (1. - (beta1**(iter)))
                    r_k_hat = R[layer][key] / (1. - (beta2**(iter)))

                    nn.model[layer][key] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + eps)
    
        # Print loss and test sample
        if iter % print_after == 0:
            print('Iter-{} loss: {:.4f}'.format(iter, loss))
            sample = nn.test(X_mini[0], state, size=100)
            print(sample)
            
    return nn

In [ ]:
# Hyper-parameters
time_step = 10 # width, minibatch size and test sample size as well
num_layers = 1 # depth
n_iter = 100 # epochs
alpha = 1e-4 # learning_rate
p_dropout = 0.95 # q=1-p, q=keep_prob and p=dropout.
print_after = 10 # n_iter//10 # print training loss, valid, and test
num_hidden_units = 64 # num_hidden_units in hidden layer
num_input_units = len(char_to_idx) # vocab_size = len(char_to_idx)

# Build the network and learning it or optimizing it using SGD
net = RNN(D=num_input_units, H=num_hidden_units, L=num_layers, char2idx=char_to_idx, idx2char=idx_to_char, 
          p_dropout=p_dropout)

# Start learning using BP-SGD-ADAM
adam_rnn(nn=net, X_train=X, y_train=y, alpha=alpha, mb_size=time_step, n_iter=n_iter, print_after=print_after)

# # Display the learning curve and losses for training, validation, and testing
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

plt.plot(net.losses['train'], label='Train loss')
plt.plot(net.losses['smooth train'], label='Train smooth loss')
plt.legend()
plt.show()

Iter-10 loss: 33.8689
ccCo oee1yysh m tanstt ocere itie twtad  nc trpeno,maot8 o,n-2s ie9Uwivh7rd h%本dice 7 uL%gEcw ifaso e
Iter-20 loss: 31.8159
ctJdothd loo sn no,lnee irh oy inthar 0fucaf  et5eapangi iphran cew,Ned aiaiC sy ivrg Jax-h, eopHoaio
Iter-30 loss: 29.9081
cpfhnibe toithoama4 wvcNan do AlsJisorthes,rf iaenoug,fmf8icnsai, 'stwe"do ol ueelen8u;biaiJat uhxtts
Iter-40 loss: 28.7452
cldiwey ,cdre alasear dhe sin oanmaIggis iota. 4hcs org ou2, g(the nfnikanatr Ud wacleahan Japeeols s
Iter-50 loss: 30.6421
ceCytldnd mimg pa as iobins w rgtea  7tamsDUy pihyca CopWers -e nfr ored as in oame Ss th eholorhe 9O
Iter-60 loss: 26.8149
cc18bthedvlny pathes'e" an rytiomiixgapenduJlawl capct'int  1prpOanlo ,enth. SIt in Iu inrAJEawo Ah i
Iter-70 loss: 28.8432
cMllkrls9inetmptureara, Tlitaous1anc l6 pat cinlparterecdand Japanscas corg an re tne eonOmin 1ehDinu
Iter-80 loss: 25.8203
ctind mord kocutbits an thc omDJapancib ann pinterr2 2f 9atin 9ce. Nue aodes .s lir, uheanjougt Wf uh
Iter-90 

Iter-670 loss: 6.7714
cind iiol.bws Aoreeit es aes uorly haka teristo s Hexoef wh1eha fers aleoh cighurtros en'st erlethag 
Iter-680 loss: 6.6781
chasory tto fecelationse makmelwopetrist Japane a rexla mdedo7ners Eiron. Serer ing che S-na joveppen
Iter-690 loss: 6.5466
cetitremdido trus -alleJasrfesilaIcicabaI wory ist o bopeaed anvole norchassl Itlo's IunoteNrfleagmCh
Iter-700 loss: 6.4739
chonst ar ris. Jopanexsgro7 mr eounthithin rid it rids , whith-inkeJapansl cigh r. Japan's uden i9.  
Iter-710 loss: 12.4927
cano lo asppofistlegin andis ohd se iot J'sed pyo levireanfs. Seateak. Iry stu,centopinpor  Dan a. ie
Iter-720 loss: 13.8442
ching lor, Ct cin  whoubou 1rth o"-nns pua leanobad Thes iito ih ixpacedasl Bfulykerim  itekhkat Sits
Iter-730 loss: 5.7799
c andioh Indes aidel alexenpu9 Hunor,iK.D femiin int un,vwI aiunorlhallote it itreapans uat copalwol 
Iter-740 loss: 5.6771
canis rhand th. Inpevesolb'se 18tylarulstircr at monalD pesaty, Kymarnest rigin "Siite 3himt. Japun a
Iter-7

Iter-1330 loss: 6.1910
catitim  fhlure Eaper,enow ,afhes lobala tomo pintoNutiag Japa , wcryag nt tintino arnk19ubth., ox an
Iter-1340 loss: 3.6628
c Gitea an is citiklechef Ja kvk po thalls tmbas of int itymby , tomut re cuna Japanecn-reapen ig Jop
Iter-1350 loss: 3.5554
ch ballcainar ce targdst. Thl ofeanebe ectiiaby whist Japle pegSean Houstiiny Japan E8 t 's te teadle
Iter-1360 loss: 3.5754
cunerushacans ped las egwand Nioleg al iome fareatan sutaoe somu th mevnhes teg lv rcall cikhest pat 
Iter-1370 loss: 6.8436
ches-lawe dollm sil otrla indivinclyapiha mobit kodid fodsopun Emgiso ma legeoty f dt aons fopercunst
Iter-1380 loss: 4.5159
ceperte tilioti isst aele pnsindny pas roriapl fgiole. endat, rbats rit in an is a shana, nollen"lkle
Iter-1390 loss: 3.5684
ci,irty, Ghess Gndesuae ss ane tovcrnon. iy  ingtse  iem mitulopalatop ne sh canll miand iity ande wa
Iter-1400 loss: 3.5114
ci, tle ecpan me iturtr Wirll the an in ergHinsfbus tioluthains Japev sren r,
. Japanelso8iny tiiete 


Iter-1990 loss: 5.1367
cutircd, ehitgcenf-taedpou18gmer intowian ang oungr hame tnrlhat ed. St. th creccd stusotigf of Jaaln
Iter-2000 loss: 5.0027
cind pourorilB4cin an astigithe antiog ohaly Sounttorl Japlenle ted tinternaml ahe ngrst onalecanan s
Iter-2010 loss: 2.8104
chy with and5 Taly tie pangmrt caed the Seafony pumetsre in tht Iodika ontike feus tor lny ban m. it 
Iter-2020 loss: 2.8996
ctungld Sor onte ial Japanec Gat sse d thcanlt ing on "Jature 17ins fruts an a ran w. Emparesior , th
Iter-2030 loss: 4.6813
copomnityio ie topen as icsta utha ae dapep ceed rng cd of werlry 1suhth cskvounens6
. 日Searestincon 
Iter-2040 loss: 5.1861
ctunbly no th on les th che Gart5 ti-lhth-cobe csgtre , aa pont rovd ne iiat n te 1sr M-lerfire pentr
Iter-2050 loss: 2.9496
chislllecovincas ts eaaipst ohan acetloulich-s rgdsea aht rohare tard om afamdolaneming Jatlueg18vens
Iter-2060 loss: 4.9327
c Ganom. Dwkrd shetegsole aintr care lor1eH.ssifacl wte rehf beg lre wssln tleat intl ind eantsarevol


Iter-2650 loss: 2.8753
ctio ll a d th poreg st a sthertindind fe inhan me shd an it rle andid a ncal thed an insrwht be w me
Iter-2660 loss: 2.6945
come6ty it the East aclth nhth for. al wasl ea pory 20 eusend pan lopan aldacrche pfge ta and K it al
Iter-2670 loss: 2.7210
culicalul tirt so nhes cherod dsat ror lo lorterepko Ienle todut re ite widthe ral m itorad's ecrurd 
Iter-2680 loss: 2.6085
cted legislem cheta, male8i85 ait  a x aplie cits phd ldealatones Fane thet re tas nd la en D pry for
Iter-2690 loss: 2.6400
cind louooncakoas tol teto pone wox ld alacte to caf rory wors ase ouut Jeperrsta  mat inte on rty in
Iter-2700 loss: 3.8398
che world D5endobe m t anpiodin  printin the Easeitan Eisity im thit , Kloa weoteopalime n turarl the
Iter-2710 loss: 2.7350
conomy by par thve as tedes a nimured len moret ol the  Sonsis Japan. 18ogicghinsl r. Japesind ped th
Iter-2720 loss: 2.6993
che Wortd a 1 utremocalmainslecaAlis of peo, iht hd thal al mant on 12 te ault anet art csin Hhe sers


...

Iter-4630 loss: 2.7944
ccymsile anpangSitlacce bo tod in Subity, it of mrn as insin afginne plochushd lanfrus eo tat  the ig
Iter-4640 loss: 5.2520
catit tintoletirs ch-lse popeec l in the s.  a timpist inrl bh prbol oaskutsreg rteciK oca nsttrke m 
Iter-4650 loss: 3.9259
cted legislin Alearealinsiona iss bo itoker ss to fet r94alic. -本L Hinsof eountnF so tenpl "o ark pun
Iter-4660 loss: 2.8081
catit . th-ta pert esRa syin ridco, etomy ttod  nereraul the aduins opoleta totwoo tuth cann reumolog
Iter-4670 loss: 2.7501
cy ithe wrtury wo loba powesransy-g ahesv lertan tee cacd ensinf in thin t ront renthns ahe adthe mlo
Iter-4680 loss: 2.8476
cy, the thily ciflec riry imp n  tinelshkk. Japad's s crta n renalloo loma inalecallD bar take uiit 1
Iter-4690 loss: 2.7056
cand as ind p4ition wf far D.eghe aloredpan, atlone bofiontr rsl catl testmompene thin lf iotand sffo
Iter-4700 loss: 2.7133
ctyo al m rtho tha ion lreelotety 9sg the at Actalde nactan w tlowateagc ta the rLopeiiotin frcgen ar


Iter-5290 loss: 2.6142
condsitfencus dedels ey pon orillof4ldop uthal winsetonn n werns pedeke ense norl Japly20loJa. so ma 
Iter-5300 loss: 2.7152
cy, the thilt enflJapan's ofe come wes ind on in with imlerampor,ed ancxe ctorcs aid anct ant od tin 
Iter-5310 loss: 2.7707
came to an 1apa mant rd Nhilatard the war mintoredevine pusilargeste mar ha ponf bollrtarg en ar iv. 
Iter-5320 loss: 2.8160
conomy by pinitimith n mitere ae aclloslevoitakh ctun chine po ch35 me ten ma thr tan in ow7d a mid  
Iter-5330 loss: 4.4029
constitutio t. Aseapds taencaml thion. Japleclumo acanor dheie ted the ssrth-. map eas ce ity inslaog
Iter-5340 loss: 4.0794
c Games.
. le trepont Ninn percon leiioplro eine hecindui, is bafacrun rngitrendol1. Weath nl the "r 
Iter-5350 loss: 2.7056
cted legislen slgefrink we  an, hon ras the ns lnt la tiothe w nstan Chin po is tarty cse  troebap s 
Iter-5360 loss: 4.1444
chy with ane my per, isl ite odutan rn 1otains Jacele Japaniset Ind an cirime ioxthacre a utixcrChuro


Iter-5950 loss: 2.7410
camicat ca mba eatiegh in ton the map of pewein tve nar. Japasiletor pon ofre esp ope essim tomelpana
Iter-5960 loss: 4.5680
che whe bty onl the Eat. p wiot cn leatale biky, Jle8, wfew. Japun r the "itafe ts of and an orycvjwi
Iter-5970 loss: 3.9965
cind lrosl co soba  font whicak .esengestaCd whttn s siorte es mhe notprgedia. 4nd th creko. Inteke,e
Iter-5980 loss: 2.7923
ccessive fens henklame w,rcharthacalle esl the he ig the S. wof eed onkrsesWcsh ched a本 20hambiigohin
Iter-5990 loss: 2.8002
cy, the thily ist r llutseneundnsa the ntlovd plorlan regean. Japuneald'sn f ot SSinore ine penter ls
Iter-6000 loss: 2.7979
cted legislen wo lo an r lat es, mint of 1atere ap icntar-ceear, Japan we aout ma the tan tind bak Th
Iter-6010 loss: 2.5591
ccessive fens aIntuathic mercund Jasl antertae conis theesethe ,d iis ofheseg nale wes tedeat.  nf ba
Iter-6020 loss: 2.5754
ch includesa icgercLolowios anda tod,uth. n marand ote tahe sxkke lan Pvealeme warthet ly the erglers


Iter-6610 loss: 2.7078
c Games.
. ke oletolu he mo if in orcat arewan thn "prt t on lo nthe ngrlt ev. the w  tr im. 4wfcin 1
Iter-6620 loss: 2.7065
cy, the thile iot  itrende ocdin to utrnt  in the tarleialsovionimt acrkyT lothous thd Toke indevered
Iter-6630 loss: 2.6316
chy with ane le andent" tyeouthe hin th chase the l gustio, wint th tregiotyr s to in lurol th cha m 
Iter-6640 loss: 4.2543
city propercake andintore wt ind aunsra. Japan trrd ito en an rcta pceglth taein tar coy ,ondth merca
Iter-6650 loss: 3.5867
constitutio ton lrta. Japar a moth rsrcrd I. un id reapra.. the antit  rcr powint r me tnrly cs racic
Iter-6660 loss: 2.3233
cted legislin cf lapal be tunt a. kake hentha  st aeeca, of tta mhe int re moltr uoha. wh cansiv , . 
Iter-6670 loss: 2.4380
city propercake foom 1han teisy inme fedof cas oh rseas ofe feponhin regrony , frrot paercoke f 4tros
Iter-6680 loss: 2.5507
c Games.
. led theferirmsdita, tith tore ain the clenoctano41t.m t ro lothe  ata pen"ari anasar phe b


Iter-7270 loss: 2.6460
came to an tapd the hd re pan wirte eate powtle Tal the iin mbtin the SJapan wit on"lobaco itak achun
Iter-7280 loss: 2.5916
chy with and an lorad'sigs.edec tret lioocod ke inte en in tio tand de ine easle auntea eamtoty moreh
Iter-7290 loss: 2.5737
came to an 1ape am hil tertan Sis lantict iced me aedilirg otpen l redooy xcr. Dh ra timyt ion th th 
Iter-7300 loss: 5.0417
ch includeso icoleclUhu  ta mbot th th a 17t. Jad 3e itlaite agdis. Jhire aleea poJa. seaseledglat se
Iter-7310 loss: 2.6461
ch includeso ingecrasit Langictf cviithechexapina l perol tnd th beanoryI sedit. in exbat idgins rogi
Iter-7320 loss: 4.0709
ccessive fent S4sitn, Japas rchurte pidit  re ingthe SLovopnd thetiditskeviit Thi toc lithich  al on"
Iter-7330 loss: 2.6143
cand as ind Stalin t8Umatgrstl he the otlindelecetsra renaler litg runtma tin int raty inteov tiat Ja
Iter-7340 loss: 2.6126
creisl masa in hicy-sand imita l tl wes the 2exerclum 1nis She -Jatar un rchale onotee wr ur "sr mait
